In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from unidecode import unidecode

# Read in CSV here
# dataset from https://www.lineups.com/fantasy-football-stats
player_data = pd.read_csv("weeks_1_thru_8.csv")
player_data = player_data.drop(['SNAP', 'TOUCH', '#', 'RTG'], axis=1)
player_data['%'] = player_data['%'] / 100
player_data['FPPG'] = player_data['FPTS'] / player_data['GP']
player_data.rename(columns={'%': 'SNAP %'}, inplace=True)


# group the dataframe by position and get the maximum and minimum FPPG for each position
max_fppg = player_data.groupby("POS")["FPPG"].max()
min_fppg = player_data.groupby("POS")["FPPG"].min()

# define a function to map the FPPG to a rating between 65 and 99
def map_fppg_to_rating(fppg, max_fppg, min_fppg):
    rating = ((fppg - min_fppg) / (max_fppg - min_fppg)) * 34 + 65
    return int(round(rating))

# apply the mapping function to each row in the dataframe
player_data["RTG"] = player_data.apply(lambda x: map_fppg_to_rating(x["FPPG"], max_fppg[x["POS"]], min_fppg[x["POS"]]), axis=1)

# sort the dataframe by position and rating
player_data = player_data.sort_values(["POS", "RTG"], ascending=[True, False])

# print the result

print(player_data)

                     NAME POS TEAM  DEP  BYE  GP  FPTS  SNAP %      FPPG  RTG
173         Kyle Juszczyk  FB   SF    4    9   8  40.6   0.580  5.075000   99
221           Alec Ingold  FB   LV    6   11   8  23.1   0.244  2.887500   84
235        Patrick Ricard  FB  BAL    4   10   8  17.6   0.585  2.200000   79
240              C.J. Ham  FB  MIN    3    7   8  15.7   0.326  1.962500   77
271            Alex Armah  FB  WAS    6   14   7   9.2   0.125  1.314286   73
..                    ...  ..  ...  ...  ...  ..   ...     ...       ...  ...
318          Malik Taylor  WR   GB    1   10   7   2.4   0.063  0.342857   65
330           Marcus Kemp  WR   KC    2   14   9   1.8   0.057  0.200000   65
332  Ihmir Smith-Marsette  WR  MIN    2    8   4   1.6   0.053  0.400000   65
337    Phillip Dorsett II  WR  HOU    1    6   3   1.3   0.096  0.433333   65
342       Diontae Spencer  WR  DEN    2   10   8   0.7   0.073  0.087500   65

[353 rows x 10 columns]


## CHANGE WEEK HERE 

In [119]:
# Read in Def CSV here
defense_data = pd.read_csv("weeks_1_thru_8_def.csv")
team_abbreviations = {
    'Buffalo Bills': 'BUF', 'Baltimore Ravens': 'BAL', 'Los Angeles Rams': 'LAR', 
    'Philadelphia Eagles': 'PHI', 'Kansas City Chiefs': 'KC', 'Los Angeles Chargers': 'LAC', 
    'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN', 'Cincinnati Bengals': 'CIN', 
    'New York Giants': 'NYG', 'Cleveland Browns': 'CLE', 'San Francisco 49ers': 'SF',
    'Arizona Cardinals': 'ARI', 'Seattle Seahawks': 'SEA', 'New York Jets': 'NYJ', 
    'Las Vegas Raiders': 'LV', 'Jacksonville Jaguars': 'JAX', 'Atlanta Falcons': 'ATL', 
    'Detroit Lions': 'DET', 'Tennessee Titans': 'TEN', 'New England Patriots': 'NE', 
    'Denver Broncos': 'DEN', 'Dallas Cowboys': 'DAL', 'Indianapolis Colts': 'IND',
    'Green Bay Packers': 'GB', 'Houston Texans': 'HOU', 'Washington Commanders': 'WAS',
    'Tampa Bay Buccaneers': 'TB', 'New Orleans Saints': 'NO', 'Chicago Bears': 'CHI', 
    'Pittsburgh Steelers': 'PIT', 'Carolina Panthers': 'CAR', 'Washington Football Team': 'WAS'
}

# map teams to ids 
team_ids = {team_abbreviations[team]: i for i, team in enumerate(team_abbreviations.keys())}

defense_data['NAME'] = defense_data['NAME'].replace(team_abbreviations)
# define a function to calculate the new rating
def calculate_new_rating(row):
    pts_allowed = row["PTS ALLOW"]
    yds_allowed = row["YARDS ALLOW"]
    sacks = row['SACKS']
    picks = row['INT']
    fumb = row['FF']
    G = row['GP']
    # calculate def points
    total_pts = pts_allowed + (yds_allowed * 0.1) + (sacks*-0.5) + (picks*-2) + (fumb*-2)
    # calculate new rating
    new_rating = (total_pts/G)
    # return new rating
    return round(new_rating)

# apply the function to each row and create a new column "NEW RTG"
defense_data["NEW RTG"] = defense_data.apply(calculate_new_rating, axis=1)
defense_data["RTG"] = defense_data["NEW RTG"]

# find the min and max RTG values
df = defense_data
min_rating = df['RTG'].min()
max_rating = df['RTG'].max()

# apply the scaling formula to each RTG value
df['RTG'] = 67 + (df['RTG'] - min_rating) * (75 - 67) / (max_rating - min_rating)

# adjust the scaling so that the average rating is 78
average_rating = df['RTG'].mean()
df['RTG'] = df['RTG'] - average_rating
defense_data = df
defense_data = defense_data.drop(['NEW RTG',' SP TD', 'DEF TD', 'GP', 'FPTS', 'PLAYS ALLOW', 'SACKS', 'FF', 'FR', 'INT'], axis=1)
def_ratings = dict(zip(defense_data['NAME'], defense_data['RTG']))
print(defense_data["RTG"].mean())
print(defense_data)

-9.769962616701378e-15
   NAME       RTG  BYE  PTS ALLOW  YARDS ALLOW
0    NO -1.267241   14        155         2776
1   ARI -3.198276   13        155         2889
2   DEN -2.370690    9        153         2896
3   BUF -4.853448    7        118         2101
4   CAR -2.922414   13        183         2638
5    NE -1.543103   10        170         3063
6   DAL  0.939655    9        192         2972
7   CHI  0.112069   14        224         3140
8   IND  0.387931   14        213         3305
9   CIN -0.439655   10        203         3251
10  CLE -1.543103    9        196         2787
11   TB -0.439655   11        183         2686
12  MIN  1.215517    7        191         3069
13  MIA  1.767241   11        242         3527
14  PHI  0.663793    7        218         3198
15  LAR -0.715517    7        196         3134
16   SF -0.163793    9        202         2705
17  PIT -0.991379    9        169         2834
18  LAC  0.387931    8        201         2869
19  HOU  1.767241    6        258    

In [120]:
# Read in schedule CSV here
# https://fixturedownload.com/sport/american-football
schedule_data = pd.read_csv("nfl_2021_schedule.csv")
schedule_data = schedule_data.drop(['Match Number', 'Location', 'Result'], axis=1)

schedule_data['Home Team'] = schedule_data['Home Team'].replace(team_abbreviations)
schedule_data['Away Team'] = schedule_data['Away Team'].replace(team_abbreviations)
print(schedule_data)

          Round Number              Date Home Team Away Team
0                    1  09/09/2021 20:20        TB       DAL
1                    1  12/09/2021 13:00       HOU       JAX
2                    1  12/09/2021 13:00       WAS       LAC
3                    1  12/09/2021 13:00       IND       SEA
4                    1  12/09/2021 13:00       CAR       NYJ
..                 ...               ...       ...       ...
280   Divisional Round  23/01/2022 15:00        TB       LAR
281   Divisional Round  23/01/2022 18:30        KC       BUF
282  Conference Champs  30/01/2022 15:00        KC       CIN
283  Conference Champs  30/01/2022 18:30       LAR        SF
284         Super Bowl  13/02/2022 18:30       CIN       LAR

[285 rows x 4 columns]


## CHANGE WEEK HERE 

In [121]:
# extract week ____ schedule
week_schedule = schedule_data[schedule_data['Round Number'] == "9"]

# create a dictionary of teams and their opponents in week 7
teams_week = dict(zip(week_schedule['Home Team'], week_schedule['Away Team']))
teams_week.update(dict(zip(week_schedule['Away Team'], week_schedule['Home Team'])))

# map the opponent to each player based on their team
player_data['OPP'] = player_data['TEAM'].map(teams_week)
player_data.insert(loc=4, column='TEAM_ID', value=player_data['TEAM'].map(team_ids))
player_data.insert(loc=11, column='OPP_ID', value=player_data['OPP'].map(team_ids))

In [122]:
merged_data = pd.merge(player_data, defense_data, left_on='OPP', right_on='NAME', how='left')
#player_data_week7 = merged_data
#print(player_data_week7)

In [123]:
#merged_data = pd.merge(player_data, defense_data, left_on='OPP', right_on='NAME', how='left')
merged_data = merged_data.drop(['NAME_y', 'BYE_y', 'BYE_x'], axis=1)
#merged_data = merged_data.drop(['NAME_y', 'BYE_y', 'BYE_x'], axis=1)
merged_data.rename(columns={'NAME_x': 'NAME', 'RTG_x': 'RTG', 'GP_x': 'GP', 'GP_y': 'DEF_GP', 'RTG_y': 'DEF_RTG'}, inplace=True)
#merged_data.rename(columns={'NAME_x': 'NAME', 'RTG_x': 'RTG', 'GP_x': 'GP', 'GP_y': 'DEF_GP', 'RTG_y': 'DEF_RTG'}, inplace=True)
merged_data = merged_data.dropna()
merged_data['NAME'] = merged_data['NAME'].str.strip()
merged_data['NAME'] = merged_data['NAME'].apply(lambda x: unidecode(x))
merged_data['NAME'] = merged_data['NAME'].str.replace('.', '')
merged_data['NAME'] = merged_data['NAME'].str.replace('-', '')
                                                      
suffixes = ['Jr', 'Sr', 'II', 'III', 'IV']
suffix_regex = '|'.join(suffixes)

merged_data['NAME'] = merged_data['NAME'].str.replace(f'\\b({suffix_regex})\\b', '', regex=True).str.strip()
merged_data['NAME'] = merged_data['NAME'].str.upper()
print(merged_data)

                    NAME POS TEAM  DEP  TEAM_ID  GP  FPTS  SNAP %      FPPG  \
0          KYLE JUSZCZYK  FB   SF    4       11   8  40.6   0.580  5.075000   
1            ALEC INGOLD  FB   LV    6       15   8  23.1   0.244  2.887500   
2         PATRICK RICARD  FB  BAL    4        1   8  17.6   0.585  2.200000   
3                 CJ HAM  FB  MIN    3        7   8  15.7   0.326  1.962500   
4             ALEX ARMAH  FB  WAS    6       32   7   9.2   0.125  1.314286   
..                   ...  ..  ...  ...      ...  ..   ...     ...       ...   
347     GUNNER OLSZEWSKI  WR   NE    2       20   9   3.2   0.072  0.355556   
348         MALIK TAYLOR  WR   GB    1       24   7   2.4   0.063  0.342857   
349          MARCUS KEMP  WR   KC    2        4   9   1.8   0.057  0.200000   
350  IHMIR SMITHMARSETTE  WR  MIN    2        7   4   1.6   0.053  0.400000   
352      DIONTAE SPENCER  WR  DEN    2       21   8   0.7   0.073  0.087500   

     RTG  OPP_ID  OPP   DEF_RTG  PTS ALLOW  YARDS A

/var/folders/yb/zh79qwpj2dq65z31rw7dndsc0000gn/T/ipykernel_26773/1555091681.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_data['NAME'] = merged_data['NAME'].str.replace('.', '')


In [124]:
#merged_data.to_csv('week7_data.csv', index=False)

# http://rotoguru1.com/cgi-bin/fyday.pl?game=dk
# could be huge

## CHANGE WEEK HERE 

In [1]:
salary_data = pd.read_csv("week_9_draftkings_2021.csv", sep=';')
salary_data['Team'] = salary_data['Team'].str.upper()
salary_data['Oppt'] = salary_data['Oppt'].str.upper()
salary_data['Pos'] = salary_data['Pos'].str.upper()
salary_data = salary_data.drop(['Year', 'GID'], axis=1)
print(salary_data)

def_pt_data = salary_data[salary_data['Pos'] == 'DEF']
print(def_pt_data)

NameError: name 'pd' is not defined

In [126]:
# new lists for mapping
list1 = ['CIN', 'LAR', 'MIA', 'NWE', 'GNB', 'PHI', 'TAM', 'BAL', 'LVR', 'ARI', 'ATL', 'WAS', 'IND', 'TEN', 'NYG', 'NOR', 'DEN', 'DET', 'CLE', 'SEA', 'NYJ', 'KAN', 'SFO', 'CHI', 'HOU', 'CAR', 'BUF', 'DAL', 'LAC', 'PIT', 'JAC', 'MIN']
list1.sort()
list2 = ['TB', 'HOU', 'WAS', 'IND', 'CAR', 'CIN', 'TEN', 'DET', 'BUF', 'ATL', 'KC', 'NO', 'NYG', 'NE', 'LAR', 'LV', 'NYJ', 'JAX', 'MIA', 'PHI', 'PIT', 'CHI', 'CLE', 'ARI', 'SEA', 'LAC', 'BAL', 'GB', 'DEN', 'MIN', 'SF', 'DAL']
list2.sort()

# flop patriots and saints for mapping
e = list1[21]
list1[21] = list1[22]
list1[22] = e

# Replace the elements in s with the corresponding elements in list2 if they are different
salary_data['Team'].replace(dict(zip(list1, list2)), inplace=True)
salary_data['Oppt'].replace(dict(zip(list1, list2)), inplace=True)

# change name format
salary_data['Name'] = salary_data['Name'].apply(lambda x: unidecode(x))
salary_data['Name'] = salary_data['Name'].str.strip()
salary_data['Name'] = salary_data['Name'].apply(lambda x: ' '.join(reversed(x.split(', '))))
salary_data['Name'] = salary_data['Name'].str.replace('.', '')
salary_data['Name'] = salary_data['Name'].str.replace('-', '')
salary_data['Name'] = salary_data['Name'].str.replace(f'\\b({suffix_regex})\\b', '', regex=True).str.strip()
salary_data['Name'] = salary_data['Name'].str.upper()
#salary_data['Name'] = salary_data['Name'].apply(lambda x: re.sub(r'^(\w+), (\w+)$', r'\2 \1', x))


# calc avg def salary and remove def from pool
avg_def_salary = salary_data.groupby('Pos')['DK salary'].mean()['DEF']
salary_data = salary_data[salary_data['Pos'] != 'DEF']
salary_data = salary_data.rename(columns={'Team': 'TEAM', 'Oppt': 'OPP', 'Pos':'POS'})
#salary_data = salary_data.drop(['Team', 'Oppt', 'Pos', 'Week'], axis=1)
salary_data = salary_data.drop(['Week'], axis=1)
salary_data = salary_data.iloc[:, [0, 1, 3, 2, 4, 6, 5]]
salary_data['h/a'] = salary_data['h/a'].replace({'h': 'home', 'a': 'away'})
salary_data = salary_data.rename(columns={'Name': 'NAME'})
salary_data = salary_data.drop(['OPP'], axis=1)

# Print the updated series
print(salary_data)
print("AVG DEFENSE SALARY")
print(avg_def_salary)

                NAME POS   h/a TEAM  DK salary  DK points
0    PATRICK MAHOMES  QB  away   KC       7200      39.24
1       DAK PRESCOTT  QB  home  DAL       6900      26.34
2         JOSH ALLEN  QB  away  BUF       7900      24.94
3     TREVOR SIEMIAN  QB  away   NO       5200      19.92
4          MAC JONES  QB  home   NE       5300      19.92
..               ...  ..   ...  ...        ...        ...
355  COLBY PARKINSON  TE  away  SEA       2500       0.00
356  CHARLIE WOERNER  TE  home   SF       2500       0.00
357     LUKE FARRELL  TE  away  JAX       2500       0.00
358     TREVON WESCO  TE  home  NYJ       2500       0.00
359     ROSS DWELLEY  TE  home   SF       2500       0.00

[360 rows x 6 columns]
AVG DEFENSE SALARY
2878.5714285714284


/var/folders/yb/zh79qwpj2dq65z31rw7dndsc0000gn/T/ipykernel_26773/1794335645.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary_data['Name'] = salary_data['Name'].str.replace('.', '')


In [127]:
# create the two dataframes
df1 = merged_data
df2 = salary_data

# get the names that are in df1 but not in df2
names_in_df1_only = df1[~df1['NAME'].isin(df2['NAME'])]['NAME'].sort_values()

# get the names that are in df2 but not in df1
names_in_df2_only = df2[~df2['NAME'].isin(df1['NAME'])]['NAME'].sort_values()

# print the results
print('Names in df1 only:', list(names_in_df1_only))
print('Names in df2 only:', list(names_in_df2_only))

final_df = merged_data.merge(salary_data, on='NAME', how='right')

final_df['POS_x'] = final_df['POS_x'].fillna(final_df['POS_y'])
final_df['TEAM_x'] = final_df['TEAM_x'].fillna(final_df['TEAM_y'])

final_df = final_df.drop(columns=['POS_y'])
final_df = final_df.drop(columns=['TEAM_y'])
final_df = final_df.drop(columns=['TEAM_ID'])
final_df = final_df.drop(columns=['OPP_ID'])
final_df = final_df.drop(columns=['SNAP %'])
final_df = final_df.rename(columns={'POS_x': 'POS'})
final_df = final_df.rename(columns={'TEAM_x': 'TEAM'})
#final_df['POS'] = final_df['POS'].replace('FB', 'RB')

final_df['OPP'] = final_df['TEAM'].map(teams_week)
final_df['DEF_RTG'] = final_df['OPP'].map(def_ratings)
df_sorted = final_df.sort_values('DK salary', ascending=False)

nan_df = df_sorted[df_sorted.isna().any(axis=1)]

high_salary_df = nan_df[nan_df['DK salary'] >= 4500]
print(high_salary_df)

print(df_sorted)

Names in df1 only: ['ALVIN KAMARA', 'ANTHONY MCFARLAND', 'ANTHONY MILLER', 'BLAINE GABBERT', 'CALVIN RIDLEY', 'CHASE CLAYPOOL', 'CHASE EDMONDS', 'CJ BEATHARD', 'CLYDE EDWARDSHELAIRE', 'CURTIS SAMUEL', 'DAMIEN HARRIS', 'DARRYNTON EVANS', 'DAVID BLOUGH', 'DEANDRE HOPKINS', 'DEE ESKRIDGE', 'DEMETRIC FELTON', 'DENZEL MIMS', 'DERRICK HENRY', 'DEVANTE PARKER', 'DJ CHARK', 'DREW LOCK', 'EQUANIMEOUS ST BROWN', 'GARDNER MINSHEW', 'GENO SMITH', 'GUNNER OLSZEWSKI', 'IHMIR SMITHMARSETTE', 'JAKE FUNK', 'JAMAAL WILLIAMS', 'JAMEIS WINSTON', 'JAMES PROCHE', 'JAMYCAL HASTY', 'JASON CABINDA', 'JODY FORTSON', 'JOHN KELLY', 'JOHNNY MUNDT', 'JONATHAN WARD', 'JONNU SMITH', 'JORDAN LOVE', 'JOSH REYNOLDS', 'JOSHUA PALMER', 'JUJU SMITHSCHUSTER', 'KEITH KIRKWOOD', 'KENNETH GAINWELL', 'KEVIN RADER', 'KIRK MERRITT', 'KJ HAMLER', 'KYLER MURRAY', "LA'MICAL PERINE", "LIL'JORDAN HUMPHREY", 'LOGAN THOMAS', 'MIKE BOONE', 'MIKE STRACHAN', 'MILES SANDERS', 'MITCHELL TRUBISKY', 'NATHAN PETERMAN', 'NICK CHUBB', 'NICK WESTB

In [128]:
# create a dictionary of the salary ranges and their corresponding labels
bins = [0, 3000, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 10000]
labels = ['<3000','3000-4000', '4000-4500', '4500-5000', '5000-5500', '5500-6000', '6000-6500', '6500-7000', '7000-7500', '7500-8000', '8000-8500', '8500-10000']

# create a new column with the salary range label for each row
grouped = final_df.groupby("POS")

# create a dictionary of dataframes, where each key is a position and the value is a dataframe containing only rows with that position
dfs_by_position = {pos: group for pos, group in grouped}

# access the dataframe for a specific position, for example, RB:
rb_df = dfs_by_position["RB"]
qb_df = dfs_by_position["QB"]
wr_df = dfs_by_position["WR"]
te_df = dfs_by_position["TE"]
fb_df = dfs_by_position["FB"]

# print QB salary stats
qb_df['Salary Range'] = pd.cut(qb_df['DK salary'], bins=bins, labels=labels)
# calculate the average RTG for each salary range
result = qb_df.groupby('Salary Range')['RTG'].mean()
print("QB SALARY DATA:")
print(result)
print()

# print RB salary stats
rb_df['Salary Range'] = pd.cut(rb_df['DK salary'], bins=bins, labels=labels)
# calculate the average RTG for each salary range
result = rb_df.groupby('Salary Range')['RTG'].mean()
print("RB SALARY DATA:")
print(result)
print()

# print WR salary stats
wr_df['Salary Range'] = pd.cut(wr_df['DK salary'], bins=bins, labels=labels)
# calculate the average RTG for each salary range
result = wr_df.groupby('Salary Range')['RTG'].mean()
print("WR SALARY DATA:")
print(result)
print()

# print TE salary stats
te_df['Salary Range'] = pd.cut(te_df['DK salary'], bins=bins, labels=labels)
# calculate the average RTG for each salary range
result = te_df.groupby('Salary Range')['RTG'].mean()
print("TE SALARY DATA:")
print(result)
print()

# print FB salary stats
fb_df['Salary Range'] = pd.cut(fb_df['DK salary'], bins=bins, labels=labels)
# calculate the average RTG for each salary range
result = fb_df.groupby('Salary Range')['RTG'].mean()
print("FB SALARY DATA:")
print(result)
print()

qb_rtg_replacements = {
    '<3000': 65,
    '3000-4000':70, 
    '4000-4500':71, 
    '4500-5000':82, 
    '5000-5500':80, 
    '5500-6000':85, 
    '6000-6500':91, 
    '6500-7000':92, 
    '7000-7500':93, 
    '7500-8000':96, 
    '8000-8500':98, 
    '8500-10000':99
}

rb_rtg_replacements = {
    '<3000': 65,
    '3000-4000':67, 
    '4000-4500':71, 
    '4500-5000':73, 
    '5000-5500':77, 
    '5500-6000':81, 
    '6000-6500':83, 
    '6500-7000':87, 
    '7000-7500':88, 
    '7500-8000':89, 
    '8000-8500':90, 
    '8500-10000':95
}

fb_rtg_replacements = {
    '<3000': 65,
    '3000-4000':75, 
    '4000-4500':75, 
    '4500-5000':75, 
    '5000-5500':75, 
    '5500-6000':75, 
    '6000-6500':75, 
    '6500-7000':75, 
    '7000-7500':75, 
    '7500-8000':75, 
    '8000-8500':75, 
    '8500-10000':75
}

wr_rtg_replacements = {
    '<3000': 65,
    '3000-4000':69, 
    '4000-4500':73, 
    '4500-5000':75, 
    '5000-5500':78, 
    '5500-6000':80, 
    '6000-6500':82, 
    '6500-7000':85, 
    '7000-7500':88, 
    '7500-8000':89, 
    '8000-8500':91, 
    '8500-10000':98
}

te_rtg_replacements = {
    '<3000': 65,
    '3000-4000':73, 
    '4000-4500':84, 
    '4500-5000':86, 
    '5000-5500':88, 
    '5500-6000':90, 
    '6000-6500':93, 
    '6500-7000':97, 
    '7000-7500':98, 
    '7500-8000':99, 
    '8000-8500':99, 
    '8500-10000':99
}

QB SALARY DATA:
Salary Range
<3000               NaN
3000-4000     66.000000
4000-4500           NaN
4500-5000     71.000000
5000-5500     84.125000
5500-6000     89.000000
6000-6500     91.000000
6500-7000     93.333333
7000-7500     93.333333
7500-8000     98.500000
8000-8500           NaN
8500-10000          NaN
Name: RTG, dtype: float64

RB SALARY DATA:
Salary Range
<3000               NaN
3000-4000     68.058824
4000-4500     72.250000
4500-5000     74.250000
5000-5500     79.666667
5500-6000     82.666667
6000-6500     84.666667
6500-7000     89.000000
7000-7500           NaN
7500-8000     90.000000
8000-8500     91.500000
8500-10000          NaN
Name: RTG, dtype: float64

WR SALARY DATA:
Salary Range
<3000         67.346154
3000-4000     71.695652
4000-4500     74.500000
4500-5000     77.090909
5000-5500     80.200000
5500-6000     80.000000
6000-6500     85.000000
6500-7000     86.750000
7000-7500     88.333333
7500-8000     87.200000
8000-8500           NaN
8500-10000    99.00

In [131]:
qb_df['Salary Range'] = qb_df['Salary Range'].astype(str)
rb_df['Salary Range'] = rb_df['Salary Range'].astype(str)
wr_df['Salary Range'] = wr_df['Salary Range'].astype(str)
te_df['Salary Range'] = te_df['Salary Range'].astype(str)
fb_df['Salary Range'] = fb_df['Salary Range'].astype(str)

qb_df.loc[qb_df['RTG'].notnull(), 'Salary Range'] = ''
mask = qb_df['Salary Range'].ne('')
qb_df.loc[mask, 'RTG'] = qb_df.loc[mask, 'Salary Range'].map(qb_rtg_replacements)

rb_df.loc[rb_df['RTG'].notnull(), 'Salary Range'] = ''
mask = rb_df['Salary Range'].ne('')
rb_df.loc[mask, 'RTG'] = rb_df.loc[mask, 'Salary Range'].map(rb_rtg_replacements)

wr_df.loc[wr_df['RTG'].notnull(), 'Salary Range'] = ''
mask = wr_df['Salary Range'].ne('')
wr_df.loc[mask, 'RTG'] = wr_df.loc[mask, 'Salary Range'].map(wr_rtg_replacements)

te_df.loc[te_df['RTG'].notnull(), 'Salary Range'] = ''
mask = te_df['Salary Range'].ne('')
te_df.loc[mask, 'RTG'] = te_df.loc[mask, 'Salary Range'].map(te_rtg_replacements)

fb_df.loc[fb_df['RTG'].notnull(), 'Salary Range'] = ''
mask = fb_df['Salary Range'].ne('')
fb_df.loc[mask, 'RTG'] = fb_df.loc[mask, 'Salary Range'].map(fb_rtg_replacements)

final_df = pd.concat([rb_df, qb_df, wr_df, te_df, fb_df], ignore_index=False)

final_df = final_df.drop(columns=['Salary Range'])
final_df = final_df.drop(columns=['DEP', 'GP', 'FPTS'])
final_df = final_df.sort_values('DK salary', ascending=False)
final_df = final_df.drop_duplicates(subset=['NAME'], keep='first')
final_df = final_df.drop(final_df[final_df['DK salary'] == 0].index)

# select the rows with pos="FB" and rtg>=73, and set their RTG values to 72
final_df.loc[(final_df["POS"] == "FB") & (final_df["RTG"] >= 73), "RTG"] = 72

# drop final columns
final_df = final_df.drop(columns=['FPPG', 'PTS ALLOW', 'YARDS ALLOW'])

# print the result
final_df.to_csv('week9_data.csv', index=False)
print(final_df)


                    NAME POS TEAM   RTG  OPP   DEF_RTG   h/a  DK salary  \
141          COOPER KUPP  WR  LAR  99.0   SF -0.163793  away       9500   
46   CHRISTIAN MCCAFFREY  RB  CAR  89.0  ARI -3.198276  away       8400   
43       JONATHAN TAYLOR  RB  IND  94.0  JAX  1.491379  home       8100   
50           DALVIN COOK  RB  MIN  86.0  LAC  0.387931  away       8000   
153        DAVANTE ADAMS  WR   GB  90.0  SEA  1.215517  home       7900   
..                   ...  ..  ...   ...  ...       ...   ...        ...   
336          ANDREW BECK  TE  DEN  65.0  PHI  0.663793  home       2500   
337          JOSH OLIVER  TE  BAL  68.0  MIA  1.767241  away       2500   
308          WILL DISSLY  TE  SEA  72.0   GB -1.543103  away       2500   
339     JAMES WINCHESTER  TE   KC  65.0   LV -0.439655  away       2500   
341        CETHAN CARTER  TE  MIA  65.0  BAL  0.939655  home       2500   

     DK points  
141       26.2  
46        26.1  
43        27.6  
50        20.8  
153       14.8

In [130]:
nan_df = final_df[final_df.isna().any(axis=1)]
print(nan_df)

Empty DataFrame
Columns: [NAME, POS, TEAM, RTG, OPP, DEF_RTG, h/a, DK salary, DK points]
Index: []
